# Module 3. Deployment on MMS(Multi Model Server)
---

본 모듈에서는 모델의 배포(deployment)를 수행합니다. 노트북 실행에는 약 15분 가량 소요되며, 핸즈온 실습 시에는 25분을 권장드립니다.

<br>

## 1. Inference script
---

아래 코드 셀은 `src` 디렉토리에 SageMaker 추론 스크립트인 `inference.py`를 저장합니다.<br>

이 스크립트는 SageMaker 상에서 MMS(Multi Model Server)를 쉽고 편하게 배포할 수 이는 high-level 툴킷인 SageMaker inference toolkit의 인터페이스를
사용하고 있으며, 여러분께서는 인터페이스에 정의된 핸들러(handler) 함수들만 구현하시면 됩니다.

#### MMS(Multi Model Server)란?
- [https://github.com/awslabs/multi-model-server](https://github.com/awslabs/multi-model-server) (2017년 12월 초 MXNet 1.0 릴리스 시 최초 공개, MXNet용 모델 서버로 시작)
- Prerequisites: Java 8, MXNet (단, MXNet 사용 시에만)
- MMS는 프레임워크에 구애받지 않도록 설계되었기 때문에, 모든 프레임워크의 백엔드 엔진 역할을 할 수 있는 충분한 유연성을 제공합니다.
- SageMaker MXNet 추론 컨테이너는 SageMaker inference toolkit으로 MMS를 래핑하여 사용합니다.

In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
%%writefile ./src/inference.py

import os
import pandas as pd
import gluonts 
import numpy as np
import argparse
import json
import pathlib
from mxnet import gpu, cpu
from mxnet.context import num_gpus
import matplotlib.pyplot as plt

from gluonts.dataset.util import to_pandas
from gluonts.mx.distribution import DistributionOutput, StudentTOutput, NegativeBinomialOutput, GaussianOutput
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions, backtest_metrics
from gluonts.model.predictor import Predictor
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset


def model_fn(model_dir):
    path = pathlib.Path(model_dir)   
    predictor = Predictor.deserialize(path)
    print("model was loaded successfully")
    return predictor


def transform_fn(model, request_body, content_type='application/json', accept_type='application/json'):

    data = json.loads(request_body)
    target_test_df = pd.DataFrame(data['value'], index=data['timestamp'])
        
    target = target_test_df.values
    num_series = target_test_df.shape[1]
    start_dt = target_test_df.index[0]
    print(f"num_series: {num_series}, start_dt: {start_dt}")
    test_lst = []
    
    for i in range(0, num_series):
        target_vec = target[:, i]
        dic = {FieldName.TARGET: target_vec, 
               FieldName.START: start_dt} 
        test_lst.append(dic)
        
    print("test_lst: \n", test_lst)
    test_ds = ListDataset(test_lst, freq='1D')

    
    response_body = {}
    forecast_it = model.predict(test_ds)
    for idx, f in enumerate(forecast_it):
        response_body[f'item_{idx}'] = f.samples.mean(axis=0).tolist()

    return json.dumps(response_body)

Overwriting ./src/inference.py


<br>

## 2. Test Inference code 
---

엔드포인트 배포 전, 추론 스크립트를 검증합니다. 

In [17]:
%store -r

### 2.1. 테스트 데이터 준비

In [18]:
from src.inference import model_fn, transform_fn
import json
import numpy as np
import pandas as pd

# Prepare test data
target_test_df = pd.read_csv("data/target_train.csv")
target_test_df.set_index(target_test_df.columns[0], inplace=True)


num_series = 3
pred_length = 28
target_test_df = target_test_df.iloc[-pred_length:,:num_series]
input_data = {'value': target_test_df.values.tolist(), 
              'timestamp': target_test_df.index.tolist()}


# print("input_data: \n", input_data)

request_body = json.dumps(input_data)

print("request_body: \n", request_body)



request_body: 
 {"value": [[12.7, 35.6, 26.1], [17.3, 45.8, 24.8], [17.7, 49.4, 29.4], [16.2, 47.7, 29.7], [19.1, 55.4, 33.6], [23.4, 58.1, 36.3], [22.4, 57.4, 38.4], [15.0, 41.3, 26.0], [17.3, 41.4, 28.3], [16.2, 48.6, 28.7], [18.9, 49.0, 31.5], [19.8, 56.4, 35.3], [21.3, 56.9, 36.6], [21.5, 60.0, 36.6], [15.8, 37.5, 28.1], [16.2, 42.2, 30.4], [16.5, 44.0, 28.6], [17.8, 48.8, 31.4], [19.7, 54.7, 35.8], [19.0, 57.0, 33.1], [23.4, 55.5, 34.2], [13.6, 42.8, 24.6], [16.5, 49.8, 30.0], [15.4, 50.0, 28.2], [17.8, 49.5, 29.8], [20.7, 51.5, 33.2], [23.6, 53.8, 34.2], [22.7, 63.4, 40.2]], "timestamp": ["2017-12-04", "2017-12-05", "2017-12-06", "2017-12-07", "2017-12-08", "2017-12-09", "2017-12-10", "2017-12-11", "2017-12-12", "2017-12-13", "2017-12-14", "2017-12-15", "2017-12-16", "2017-12-17", "2017-12-18", "2017-12-19", "2017-12-20", "2017-12-21", "2017-12-22", "2017-12-23", "2017-12-24", "2017-12-25", "2017-12-26", "2017-12-27", "2017-12-28", "2017-12-29", "2017-12-30", "2017-12-31"]}


### 2.2. 모델 로딩 함수 테스트

In [19]:
# Test inference script 
model = model_fn('./model')


model was loaded successfully


### 2.3. 모델 추론 함수 테스트

In [21]:
response = transform_fn(model, request_body)


num_series: 3, start_dt: 2017-12-04
test_lst: 
 [{'target': array([12.7, 17.3, 17.7, 16.2, 19.1, 23.4, 22.4, 15. , 17.3, 16.2, 18.9,
       19.8, 21.3, 21.5, 15.8, 16.2, 16.5, 17.8, 19.7, 19. , 23.4, 13.6,
       16.5, 15.4, 17.8, 20.7, 23.6, 22.7]), 'start': '2017-12-04'}, {'target': array([35.6, 45.8, 49.4, 47.7, 55.4, 58.1, 57.4, 41.3, 41.4, 48.6, 49. ,
       56.4, 56.9, 60. , 37.5, 42.2, 44. , 48.8, 54.7, 57. , 55.5, 42.8,
       49.8, 50. , 49.5, 51.5, 53.8, 63.4]), 'start': '2017-12-04'}, {'target': array([26.1, 24.8, 29.4, 29.7, 33.6, 36.3, 38.4, 26. , 28.3, 28.7, 31.5,
       35.3, 36.6, 36.6, 28.1, 30.4, 28.6, 31.4, 35.8, 33.1, 34.2, 24.6,
       30. , 28.2, 29.8, 33.2, 34.2, 40.2]), 'start': '2017-12-04'}]


In [24]:
outputs = json.loads(response)
print(outputs['item_0'])

[16.444486618041992, 19.705787658691406, 19.8192081451416, 21.79358673095703, 23.132600784301758, 24.98563003540039, 25.902881622314453, 17.588369369506836, 20.195985794067383, 20.150680541992188, 21.783811569213867, 22.881406784057617, 24.798629760742188, 25.934206008911133, 17.154808044433594, 19.984914779663086, 19.66559600830078, 21.007383346557617, 22.369077682495117, 23.784221649169922, 25.097583770751953, 16.612205505371094, 19.1953067779541, 18.974210739135742, 20.82592010498047, 21.745948791503906, 23.698139190673828, 24.192283630371094]


<br>

## 3. Local Endpoint Inference
---

충분한 검증 및 테스트 없이 훈련된 모델을 곧바로 실제 운영 환경에 배포하기에는 많은 위험 요소들이 있습니다. 따라서, 로컬 모드를 사용하여 실제 운영 환경에 배포하기 위한 추론 인스턴스를 시작하기 전에 노트북 인스턴스의 로컬 환경에서 모델을 배포하는 것을 권장합니다. 이를 로컬 모드 엔드포인트(Local Mode Endpoint)라고 합니다.

In [25]:
import os
import time
import sagemaker
from sagemaker.mxnet import MXNetModel
role = sagemaker.get_execution_role()

In [26]:
local_model_path = f'file://{os.getcwd()}/model/model.tar.gz'
endpoint_name = "local-endpoint-store-item-demand-forecast-{}".format(int(time.time()))


아래 코드 셀을 실행 후, 로그를 확인해 보세요. MMS에 대한 세팅값들을 확인하실 수 있습니다.

```bash
algo-1-u3xwd_1  | MMS Home: /usr/local/lib/python3.6/site-packages
algo-1-u3xwd_1  | Current directory: /
algo-1-u3xwd_1  | Temp directory: /home/model-server/tmp
algo-1-u3xwd_1  | Number of GPUs: 0
algo-1-u3xwd_1  | Number of CPUs: 2
algo-1-u3xwd_1  | Max heap size: 878 M
algo-1-u3xwd_1  | Python executable: /usr/local/bin/python3.6
algo-1-u3xwd_1  | Config file: /etc/sagemaker-mms.properties
algo-1-u3xwd_1  | Inference address: http://0.0.0.0:8080
algo-1-u3xwd_1  | Management address: http://0.0.0.0:8080
algo-1-u3xwd_1  | Model Store: /.sagemaker/mms/models
...
```

In [27]:
local_model = MXNetModel(model_data=local_model_path,
                         role=role,
                         source_dir='src',
                         entry_point='inference.py',
                         framework_version='1.6.0',
                         py_version='py3')

predictor = local_model.deploy(instance_type='local', 
                           initial_instance_count=1, 
                           endpoint_name=endpoint_name,
                           wait=True)

Attaching to p5ye4je5hi-algo-1-8mv2q
p5ye4je5hi-algo-1-8mv2q | Collecting pandas==1.1.5
p5ye4je5hi-algo-1-8mv2q |   Downloading pandas-1.1.5-cp36-cp36m-manylinux1_x86_64.whl (9.5 MB)
     |████████████████████████████████| 9.5 MB 22.4 MB/s eta 0:00:01
p5ye4je5hi-algo-1-8mv2q | Collecting gluonts==0.6.7
p5ye4je5hi-algo-1-8mv2q |   Downloading gluonts-0.6.7-py3-none-any.whl (569 kB)
     |████████████████████████████████| 569 kB 60.3 MB/s eta 0:00:01
p5ye4je5hi-algo-1-8mv2q | Collecting toolz~=0.10
p5ye4je5hi-algo-1-8mv2q |   Downloading toolz-0.12.0-py3-none-any.whl (55 kB)
     |████████████████████████████████| 55 kB 7.2 MB/s  eta 0:00:01
p5ye4je5hi-algo-1-8mv2q | Requirement already satisfied: numpy~=1.16 in /usr/local/lib/python3.6/site-packages (from gluonts==0.6.7->-r /opt/ml/model/code/requirements.txt (line 2)) (1.17.4)
p5ye4je5hi-algo-1-8mv2q | Collecting holidays>=0.9
p5ye4je5hi-algo-1-8mv2q |   Downloading holidays-0.13-py3-none-any.whl (172 kB)
     |████████████████████████

로컬에서 컨테이너를 배포했기 때문에 컨테이너가 현재 실행 중임을 확인할 수 있습니다.

In [28]:
!docker ps

CONTAINER ID   IMAGE                                                                        COMMAND                  CREATED          STATUS          PORTS                                                 NAMES
cd80da375711   763104351884.dkr.ecr.us-east-1.amazonaws.com/mxnet-inference:1.6.0-cpu-py3   "python /usr/local/b…"   15 seconds ago   Up 14 seconds   0.0.0.0:8080->8080/tcp, :::8080->8080/tcp, 8081/tcp   p5ye4je5hi-algo-1-8mv2q
p5ye4je5hi-algo-1-8mv2q | 2022-07-17 08:26:30,525 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Generating new fontManager, this may take some time...
p5ye4je5hi-algo-1-8mv2q | 2022-07-17 08:26:30,531 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Generating new fontManager, this may take some time...
p5ye4je5hi-algo-1-8mv2q | 2022-07-17 08:26:30,540 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Generating new fontManager, this may take some time...
p5ye4je5hi-algo-1-8mv2q | 2022-07-

### Inference using SageMaker SDK

SageMaker SDK의 `predict()` 메서드로 쉽게 추론을 수행할 수 있습니다. 

In [29]:
outputs = predictor.predict(input_data)
print(outputs)

p5ye4je5hi-algo-1-8mv2q | 2022-07-17 08:26:57,205 [INFO ] W-model-14-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - num_series: 3, start_dt: 2017-12-04
p5ye4je5hi-algo-1-8mv2q | 2022-07-17 08:26:57,206 [INFO ] W-model-14-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - test_lst: 
p5ye4je5hi-algo-1-8mv2q | 2022-07-17 08:26:57,206 [INFO ] W-model-14-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle -  [{'target': array([12.7, 17.3, 17.7, 16.2, 19.1, 23.4, 22.4, 15. , 17.3, 16.2, 18.9,
p5ye4je5hi-algo-1-8mv2q | 2022-07-17 08:26:57,206 [INFO ] W-model-14-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle -        19.8, 21.3, 21.5, 15.8, 16.2, 16.5, 17.8, 19.7, 19. , 23.4, 13.6,
p5ye4je5hi-algo-1-8mv2q | 2022-07-17 08:26:57,207 [INFO ] W-model-14-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle -        16.5, 15.4, 17.8, 20.7, 23.6, 22.7]), 'start': '2017-12-04'}, {'target': array([35.6, 45.8, 49.4, 47.7, 55.4, 58.1, 57.4, 41.3, 41.4, 48.6, 49. ,
p5ye4je5hi-algo-1-8mv2q | 2022-07-17 08:26:57,

### Inference using Boto3 SDK

SageMaker SDK의 `predict()` 메서드로 추론을 수행할 수도 있지만, 이번에는 boto3의 `invoke_endpoint()` 메서드로 추론을 수행해 보겠습니다.<br>
Boto3는 서비스 레벨의 low-level SDK로, ML 실험에 초점을 맞춰 일부 기능들이 추상화된 high-level SDK인 SageMaker SDK와 달리
SageMaker API를 완벽하게 제어할 수 있습으며, 프로덕션 및 자동화 작업에 적합합니다.

참고로 `invoke_endpoint()` 호출을 위한 런타임 클라이언트 인스턴스 생성 시, 로컬 배포 모드에서는 `sagemaker.local.LocalSagemakerRuntimeClient()`를 호출해야 합니다.

In [30]:
client = sagemaker.local.LocalSagemakerClient()
runtime_client = sagemaker.local.LocalSagemakerRuntimeClient()
endpoint_name = local_model.endpoint_name

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/json',
    Accept='application/x-npy',
    Body=json.dumps(input_data)
    )
outputs = response['Body'].read().decode()

p5ye4je5hi-algo-1-8mv2q | 2022-07-17 08:27:53,500 [INFO ] W-model-8-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - num_series: 3, start_dt: 2017-12-04
p5ye4je5hi-algo-1-8mv2q | 2022-07-17 08:27:53,501 [INFO ] W-model-8-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - test_lst: 
p5ye4je5hi-algo-1-8mv2q | 2022-07-17 08:27:53,502 [INFO ] W-model-8-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle -  [{'target': array([12.7, 17.3, 17.7, 16.2, 19.1, 23.4, 22.4, 15. , 17.3, 16.2, 18.9,
p5ye4je5hi-algo-1-8mv2q | 2022-07-17 08:27:53,502 [INFO ] W-model-8-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle -        19.8, 21.3, 21.5, 15.8, 16.2, 16.5, 17.8, 19.7, 19. , 23.4, 13.6,
p5ye4je5hi-algo-1-8mv2q | 2022-07-17 08:27:53,502 [INFO ] W-model-8-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle -        16.5, 15.4, 17.8, 20.7, 23.6, 22.7]), 'start': '2017-12-04'}, {'target': array([35.6, 45.8, 49.4, 47.7, 55.4, 58.1, 57.4, 41.3, 41.4, 48.6, 49. ,
p5ye4je5hi-algo-1-8mv2q | 2022-07-17 08:27:53,502 [

In [31]:
outputs

'{"item_0": [16.57381820678711, 19.485475540161133, 20.022624969482422, 21.5930233001709, 23.302013397216797, 25.1539249420166, 25.836524963378906, 17.183427810668945, 20.263813018798828, 20.109722137451172, 22.046316146850586, 22.884986877441406, 24.5477352142334, 25.728002548217773, 17.03057289123535, 19.898099899291992, 19.560701370239258, 21.239347457885742, 22.5848388671875, 23.70880699157715, 25.67095184326172, 16.704727172851562, 19.436874389648438, 19.222536087036133, 20.789386749267578, 22.20077133178711, 23.557828903198242, 24.341289520263672], "item_1": [44.26552200317383, 52.46006393432617, 54.20823669433594, 58.337764739990234, 61.993202209472656, 64.77264404296875, 68.71761322021484, 46.06790542602539, 52.786376953125, 53.58518981933594, 56.71074676513672, 60.79315948486328, 63.52743148803711, 67.9822006225586, 44.5804443359375, 52.165836334228516, 51.97383117675781, 55.43089294433594, 58.21323776245117, 61.984745025634766, 64.7683334350586, 43.30618667602539, 50.22137832

### Local Mode Endpoint Clean-up

엔드포인트를 계속 사용하지 않는다면, 엔드포인트를 삭제해야 합니다. 
SageMaker SDK에서는 `delete_endpoint()` 메소드로 간단히 삭제할 수 있습니다.

In [32]:
def delete_endpoint(client, endpoint_name):
    response = client.describe_endpoint_config(EndpointConfigName=endpoint_name)
    model_name = response['ProductionVariants'][0]['ModelName']

    client.delete_model(ModelName=model_name)    
    client.delete_endpoint(EndpointName=endpoint_name)
    client.delete_endpoint_config(EndpointConfigName=endpoint_name)    
    
    print(f'--- Deleted model: {model_name}')
    print(f'--- Deleted endpoint: {endpoint_name}')
    print(f'--- Deleted endpoint_config: {endpoint_name}')    
delete_endpoint(client, endpoint_name)

Gracefully stopping... (press Ctrl+C again to force)
--- Deleted model: mxnet-inference-2022-07-17-08-26-12-193
--- Deleted endpoint: local-endpoint-store-item-demand-forecast-1658046368
--- Deleted endpoint_config: local-endpoint-store-item-demand-forecast-1658046368


<br>

## 4. SageMaker Hosted Endpoint Inference
---

이제 실제 운영 환경에 엔드포인트 배포를 수행해 보겠습니다. 로컬 모드 엔드포인트와 대부분의 코드가 동일하며, 모델 아티팩트 경로(`model_data`)와 인스턴스 유형(`instance_type`)만 변경해 주시면 됩니다. SageMaker가 관리하는 배포 클러스터를 프로비저닝하는 시간이 소요되기 때문에 추론 서비스를 시작하는 데에는 약 5~10분 정도 소요됩니다.

In [33]:
import os
import boto3
import sagemaker
from sagemaker.mxnet import MXNet

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)
role = sagemaker.get_execution_role()
bucket = sagemaker.Session().default_bucket()

In [34]:
model_path = os.path.join(s3_model_dir, "model.tar.gz")
endpoint_name = "endpoint-store-item-demand-forecast-{}".format(int(time.time()))

In [35]:
model = MXNetModel(model_data=model_path,
                         role=role,
                         source_dir='src',
                         entry_point='inference.py',
                         framework_version='1.6.0',
                         py_version='py3')

predictor = model.deploy(instance_type="ml.c5.large", 
                         initial_instance_count=1, 
                         endpoint_name=endpoint_name,
                         wait=True)

-----!

추론을 수행합니다. 로컬 모드의 코드와 동일합니다.

In [36]:
outputs = predictor.predict(input_data)
print(outputs)

{'item_0': [16.57381820678711, 19.485477447509766, 20.022624969482422, 21.5930233001709, 23.302013397216797, 25.1539249420166, 25.836523056030273, 17.183429718017578, 20.263813018798828, 20.109722137451172, 22.046316146850586, 22.884986877441406, 24.547731399536133, 25.728004455566406, 17.03057289123535, 19.898099899291992, 19.560701370239258, 21.239351272583008, 22.5848388671875, 23.70880699157715, 25.67095184326172, 16.704727172851562, 19.436874389648438, 19.222536087036133, 20.789386749267578, 22.200769424438477, 23.557828903198242, 24.341291427612305], 'item_1': [44.265525817871094, 52.46006393432617, 54.2082405090332, 58.3377685546875, 61.993202209472656, 64.77264404296875, 68.71761322021484, 46.06790542602539, 52.786380767822266, 53.58518981933594, 56.71074676513672, 60.79315948486328, 63.52743148803711, 67.9822006225586, 44.580448150634766, 52.165836334228516, 51.97383117675781, 55.43089294433594, 58.21323776245117, 61.984745025634766, 64.76834106445312, 43.306190490722656, 50.2

In [37]:
import boto3
client = boto3.client('sagemaker')
runtime_client = boto3.client('sagemaker-runtime')
endpoint_name = model.endpoint_name

In [38]:
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/json',
    Accept='application/json',
    Body=json.dumps(input_data)
    )
outputs = response['Body'].read().decode()

In [39]:
outputs

'{"item_0": [16.853008270263672, 19.322612762451172, 20.264875411987305, 21.6895809173584, 23.208093643188477, 24.666259765625, 25.791994094848633, 17.28667449951172, 20.07016372680664, 19.992877960205078, 21.58452606201172, 23.29920196533203, 24.273176193237305, 25.842388153076172, 17.135705947875977, 19.5638484954834, 19.59037971496582, 21.33539581298828, 22.556406021118164, 23.76131820678711, 25.470582962036133, 16.315513610839844, 19.536975860595703, 19.149011611938477, 20.34992218017578, 22.035219192504883, 23.492212295532227, 24.147998809814453], "item_1": [43.88142013549805, 52.48280715942383, 55.357688903808594, 58.26667404174805, 62.02186965942383, 65.01542663574219, 68.57644653320312, 45.89519119262695, 52.98295593261719, 53.547569274902344, 56.70088195800781, 60.956172943115234, 63.80867004394531, 67.9952621459961, 44.49686050415039, 51.725616455078125, 52.2453498840332, 55.277000427246094, 58.427616119384766, 61.6430549621582, 65.1532211303711, 43.52935028076172, 49.7991065

### SageMaker Hosted Endpoint Clean-up

엔드포인트를 계속 사용하지 않는다면, 불필요한 과금을 피하기 위해 엔드포인트를 삭제해야 합니다. 
SageMaker SDK에서는 `delete_endpoint()` 메소드로 간단히 삭제할 수 있으며, UI에서도 쉽게 삭제할 수 있습니다.

In [40]:
delete_endpoint(client, endpoint_name)

--- Deleted model: mxnet-inference-2022-07-17-08-28-26-387
--- Deleted endpoint: endpoint-store-item-demand-forecast-1658046504
--- Deleted endpoint_config: endpoint-store-item-demand-forecast-1658046504
